In [3]:
import sys
import pandas as pd
sys.path.append('/Applications/Stata/utilities')
from pystata import config
config.init('se')

from pystata import stata


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       SE—Standard Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Unlimited-user network, expiring 21 Sep 2023
Serial number: 401709301720
  Licensed to: Gabriel Dias Medeiros Pereira
               ISEG

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. Maximum number of variables is set to 5,000; see help set_maxvar.


In [4]:
df = pd.read_stata("blundbondbalanc 2.dta")
df["year"] = df["year"].dt.year

In [17]:
df

,id,ly,ln,lk,year,D83,D84,D85,D86,D87,D88,D89
0,886,4.579228,3.480440,3.254327,1982,0,0,0,0,0,0,0
1,886,4.472189,3.357781,3.346090,1983,1,0,0,0,0,0,0
2,886,4.567035,3.360103,3.485277,1984,0,1,0,0,0,0,0
3,886,4.885006,3.315239,3.667849,1985,0,0,1,0,0,0,0
4,886,4.999058,3.380052,3.831042,1986,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3931,989349,7.450092,5.983883,6.155892,1985,0,0,1,0,0,0,0
3932,989349,7.576933,5.971417,6.210421,1986,0,0,0,1,0,0,0
3933,989349,7.767560,5.923970,6.297313,1987,0,0,0,0,1,0,0
3934,989349,7.856476,6.002237,6.334439,1988,0,0,0,0,0,1,0


## Question 1)

In [5]:
stata.pdataframe_to_data(df, force=True)
stata.run(
    ''' 
    xtset id year, yearly
    reg ly l.ly ln l.ln lk l.lk D84-D89, vce(cluster id)
    estimates store POLS        
    '''
)


.  
.     xtset id year, yearly

Panel variable: id (strongly balanced)
 Time variable: year, 1982 to 1989
         Delta: 1 year

.     reg ly l.ly ln l.ln lk l.lk D84-D89, vce(cluster id)

Linear regression                               Number of obs     =      3,444
                                                F(11, 491)        =   59731.41
                                                Prob > F          =     0.0000
                                                R-squared         =     0.9942
                                                Root MSE          =     .15155

                                   (Std. err. adjusted for 492 clusters in id)
------------------------------------------------------------------------------
             |               Robust
          ly | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
          ly |
         L1. |   .9564256   .0062554   152.90

- Estimating by fixed effects

In [6]:
stata.pdataframe_to_data(df, force=True)
stata.run(
    ''' 
    xtset id year, yearly
    xtreg ly l.ly ln l.ln lk l.lk D84-D89, fe vce(cluster id)
    estimates store FE        
    '''
)


.  
.     xtset id year, yearly

Panel variable: id (strongly balanced)
 Time variable: year, 1982 to 1989
         Delta: 1 year

.     xtreg ly l.ly ln l.ln lk l.lk D84-D89, fe vce(cluster id)

Fixed-effects (within) regression               Number of obs     =      3,444
Group variable: id                              Number of groups  =        492

R-squared:                                      Obs per group:
     Within  = 0.7081                                         min =          7
     Between = 0.9941                                         avg =        7.0
     Overall = 0.9897                                         max =          7

                                                F(11,491)         =     331.59
corr(u_i, Xb) = 0.9276                          Prob > F          =     0.0000

                                   (Std. err. adjusted for 492 clusters in id)
------------------------------------------------------------------------------
             |            

- First differences

In [7]:
stata.pdataframe_to_data(df, force=True)
stata.run(
    ''' 
    xtset id year, yearly
    reg D.(ly l.ly ln l.ln lk l.lk D85-D89), nocons vce(cluster id)
    estimates store FD     
    '''
)


.  
.     xtset id year, yearly

Panel variable: id (strongly balanced)
 Time variable: year, 1982 to 1989
         Delta: 1 year

.     reg D.(ly l.ly ln l.ln lk l.lk D85-D89), nocons vce(cluster id)

Linear regression                               Number of obs     =      2,952
                                                F(10, 491)        =      92.65
                                                Prob > F          =     0.0000
                                                R-squared         =     0.3326
                                                Root MSE          =     .14912

                                   (Std. err. adjusted for 492 clusters in id)
------------------------------------------------------------------------------
             |               Robust
        D.ly | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
          ly |
         LD. |  -.0481349    .0236

- Comparing three models

In [8]:
stata.run(
    ''' 
    estimates table POLS FE FD, star(.1 .05 .01)    
    '''
)


.  
.     estimates table POLS FE FD, star(.1 .05 .01)    

--------------------------------------------------------------
    Variable |     POLS             FE              FD        
-------------+------------------------------------------------
          ly |
         L1. |  .95642564***    .55043701***                  
         LD. |                                  -.0481349**   
             |
          ln |
         --. |  .41533612***    .38556523***                  
         L1. | -.39156312***   -.22218468***                  
         D1. |                                   .3613453***  
         LD. |                                  .09486793***  
             |
          lk |
         --. |  .27799009***    .42332897***                  
         L1. | -.26537853***   -.35674603***                  
         D1. |                                  .60170257***  
         LD. |                                 -.31780568***  
             |
         D84 |  .10580755***  

## Question 2)

- Arellano and Bond GMM estimation

In [17]:
stata.run(
    ''' 
    ssc install xtabond2, replace    
    '''
)


.  
.     ssc install xtabond2, replace    
checking xtabond2 consistency and verifying not already installed...
installing into /Users/gabrieldiasmp/Library/Application Support/Stata/ado/plus
> /...
installation complete.

.     
. 


2a)
* The method used is GMM in one step with robust covariance estimation by clusters (robust)
* ly L1 is out estimation for alpha, alpha = 0.3607

In [12]:
stata.run(
    ''' 
    xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(ln lk D85-D89) gmm(l.ly) noleveleq robust    
    '''
)


.  
.     xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(ln lk D85-D89) gmm(l.ly) nol
> eveleq robust    
Favoring space over speed. To switch, type or click on mata: mata set matafavor
>  speed, perm.
  Using a generalized inverse to calculate robust weighting matrix for Hansen t
> est.
  Difference-in-Sargan/Hansen statistics may be negative.

Dynamic panel-data estimation, one-step difference GMM
------------------------------------------------------------------------------
Group variable: id                              Number of obs      =      2952
Time variable : year                            Number of groups   =       492
Number of instruments = 28                      Obs per group: min =         6
Wald chi2(0)  =         .                                      avg =      6.00
Prob > chi2   =         .                                      max =         6
------------------------------------------------------------------------------
             |               Robust
         

In [18]:
## Running equation 8 of the exercise
## Two-step GMM, endogenous variables, windmeijer correction

stata.run(
    ''' 
    xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly, lag(4 .)) gmm(ln lk, lag(3 .)) noleveleq twostep robust     
    '''
)


.  
.     xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly, lag(4 .))
>  gmm(ln lk, lag(3 .)) noleveleq twostep robust     
Favoring space over speed. To switch, type or click on mata: mata set matafavor
>  speed, perm.
  Using a generalized inverse to calculate optimal weighting matrix for two-ste
> p estimation.
  Difference-in-Sargan/Hansen statistics may be negative.

Dynamic panel-data estimation, two-step difference GMM
------------------------------------------------------------------------------
Group variable: id                              Number of obs      =      2952
Time variable : year                            Number of groups   =       492
Number of instruments = 41                      Obs per group: min =         6
Wald chi2(0)  =         .                                      avg =      6.00
Prob > chi2   =         .                                      max =         6
------------------------------------------------------------------------------
  

## Question 3)

In [19]:
##Blundell & Bond 
##one-step GMM, robust s.e., predetermined Variables 

stata.run('''
    xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly) gmm(ln lk) robust
''')


. 
.     xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly) gmm(ln lk
> ) robust
Favoring space over speed. To switch, type or click on mata: mata set matafavor
>  speed, perm.
  Using a generalized inverse to calculate robust weighting matrix for Hansen t
> est.
  Difference-in-Sargan/Hansen statistics may be negative.

Dynamic panel-data estimation, one-step system GMM
------------------------------------------------------------------------------
Group variable: id                              Number of obs      =      3444
Time variable : year                            Number of groups   =       492
Number of instruments = 101                     Obs per group: min =         7
Wald chi2(10) =  3.94e+06                                      avg =      7.00
Prob > chi2   =     0.000                                      max =         7
------------------------------------------------------------------------------
             |               Robust
          ly | Coeffici

## Examples

In [16]:
import pandas as pd
import io
import requests

data = requests.get("https://www.stata.com/python/pystata/misc/nhanes2.csv").content
nhanes2 = pd.read_csv(io.StringIO(data.decode("utf-8")))
nhanes2

,sampl,strata,psu,region,smsa,location,houssiz,sex,race,age,...,region4,smsa1,smsa2,smsa3,rural,loglead,agegrp,highlead,bmi,highbp
0,1400,1,1,S,2,1,4,Male,White,54,...,0,0,1,0,0,NaN,50-59,NaN,20.495686,0
1,1401,1,1,S,2,1,6,Female,White,41,...,0,0,1,0,0,2.564949,40-49,lead<25,21.022337,0
2,1402,1,1,S,1,1,6,Female,Other,21,...,0,1,0,0,0,NaN,20-29,NaN,24.973860,0
3,1404,1,1,S,2,1,9,Female,White,63,...,0,0,1,0,0,NaN,60-69,NaN,35.728722,1
4,1405,1,1,S,1,1,3,Female,White,64,...,0,1,0,0,0,2.995732,60-69,lead<25,27.923803,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10346,48760,32,2,MW,4,48,5,Female,White,35,...,0,0,0,1,1,NaN,30-39,NaN,20.355173,0
10347,48763,32,2,MW,4,48,2,Female,White,33,...,0,0,0,1,1,1.945910,30-39,lead<25,41.645557,1
10348,48764,32,2,MW,4,48,1,Female,White,60,...,0,0,0,1,0,NaN,60-69,NaN,35.626114,0
10349,48768,32,2,MW,4,48,1,Female,White,29,...,0,0,0,1,0,NaN,20-29,NaN,19.204464,0


In [17]:
%%stata -d nhanes2 -eret myeret

logistic highbp c.age##c.weight

ereturn list


. 
. logistic highbp c.age##c.weight

Logistic regression                                    Number of obs =  10,351
                                                       LR chi2(3)    = 2381.23
                                                       Prob > chi2   =  0.0000
Log likelihood = -5860.1512                            Pseudo R2     =  0.1689

------------------------------------------------------------------------------
      highbp | Odds ratio   Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   1.108531   .0080697    14.15   0.000     1.092827     1.12446
      weight |   1.081505    .005516    15.36   0.000     1.070748    1.092371
             |
       c.age#|
    c.weight |   .9992788   .0000977    -7.38   0.000     .9990873    .9994703
             |
       _cons |   .0002025   .0000787   -21.89   0.000     .0000946    .0004335
----------------------------------------------

In [18]:
myeret

{'e(rank)': 4.0,
 'e(N)': 10351.0,
 'e(ic)': 4.0,
 'e(k)': 4.0,
 'e(k_eq)': 1.0,
 'e(noconstant)': 0.0,
 'e(consonly)': 0.0,
 'e(k_dv)': 1.0,
 'e(converged)': 1.0,
 'e(rc)': 0.0,
 'e(k_autoCns)': 0.0,
 'e(ll)': -5860.151218806021,
 'e(k_eq_model)': 1.0,
 'e(ll_0)': -7050.765484416371,
 'e(df_m)': 3.0,
 'e(chi2)': 2381.2285312207005,
 'e(p)': 0.0,
 'e(N_cdf)': 0.0,
 'e(N_cds)': 0.0,
 'e(r2_p)': 0.16886312106704593,
 'e(cmdline)': 'logistic highbp c.age##c.weight',
 'e(cmd)': 'logistic',
 'e(predict)': 'logistic_p',
 'e(estat_cmd)': 'logit_estat',
 'e(marginsderiv)': 'default Pr',
 'e(marginsok)': 'default Pr',
 'e(marginsnotok)': 'stdp DBeta DEviance DX2 DDeviance Hat Number Residuals RStandard SCore',
 'e(title)': 'Logistic regression',
 'e(chi2type)': 'LR',
 'e(opt)': 'moptimize',
 'e(vce)': 'oim',
 'e(user)': 'mopt__logit_d2()',
 'e(crittype)': 'log likelihood',
 'e(ml_method)': 'd2',
 'e(singularHmethod)': 'm-marquardt',
 'e(technique)': 'nr',
 'e(which)': 'max',
 'e(depvar)': 'high

In [19]:
from pystata import stata

In [20]:
stata.pdataframe_to_data(nhanes2, True)

In [ ]:

# Run Stata commands in Python
stata.run('''
        didregress (satisfaction_score) (procedure), ///
                group(hospital_id) time(month)
        ''', echo=True)